# COVID-19 Face Mask Detection
### Detects when a user is not wearing a mask and displays it. 

Done by- Baibhav Saikia

### Importing the Dataset and Necessary Libraries

In [2]:
import cv2,os
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.models import load_model

In [3]:
data_path=r"C:\Users\asus\Desktop\Dataset"
folders=os.listdir(data_path)
labels=[i for i in range(len(folders))]
label_dict=dict(zip(folders,labels))
print(label_dict)

{'with mask': 0, 'without mask': 1}


### Data Preprocessing

In [4]:
img_size=100
data=[]
label=[]

for folder in folders:
    folder_path=os.path.join(data_path,folder)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
        resized=cv2.resize(gray,(img_size,img_size))
        data.append(resized)
        label.append(label_dict[folder])  

In [5]:
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
label=np.array(label)
label=np_utils.to_categorical(label)     

### Making the Neural Network Architecture

In [6]:
model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50,activation='relu'))
model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [7]:
train_data,test_data,train_label,test_label=train_test_split(data,label,test_size=0.1)

### Training and Testing the CNN

In [8]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_label,epochs=25,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/25
31/31 [==============================] - 82s 2s/step - loss: 0.7477 - accuracy: 0.5048 - val_loss: 0.6537 - val_accuracy: 0.7339
INFO:tensorflow:Assets written to: model-001.model\assets
Epoch 2/25
31/31 [==============================] - 56s 2s/step - loss: 0.6508 - accuracy: 0.6246 - val_loss: 0.4896 - val_accuracy: 0.8024
INFO:tensorflow:Assets written to: model-002.model\assets
Epoch 3/25
31/31 [==============================] - 56s 2s/step - loss: 0.4445 - accuracy: 0.8113 - val_loss: 0.2744 - val_accuracy: 0.8992
INFO:tensorflow:Assets written to: model-003.model\assets
Epoch 4/25
31/31 [==============================] - 56s 2s/step - loss: 0.2766 - accuracy: 0.9048 - val_loss: 0.2224 - val_accuracy: 0.9194
INFO:tensorflow:Assets written to: model-004.model\assets
Epoch 5/25
31/31 [==============================] - 56s 2s/step - loss: 0.2028 - accuracy: 0.9181 - val_loss: 0.1751 - val_accuracy: 0.9274
INFO:tensorflow:Assets written to: model-005.model\assets
Epoch 6/25

In [9]:
print(model.evaluate(test_data,test_label))

5/5 [==============================] - 2s 381ms/step - loss: 0.1746 - accuracy: 0.9783
[0.1745682656764984, 0.97826087474823]


### Loading the 'Best' Model

In [31]:
model = load_model('model-024.model')
classifier=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
source=cv2.VideoCapture(0)
labels_dict={1:'NO MASK'}
color_dict={1:(0,0,255)}

### Reading from the Input Source

In [ ]:
while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=classifier.detectMultiScale(gray,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)
        lab=np.argmax(result)
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[lab],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[lab],-1)
        cv2.putText(img, labels_dict[lab], (x, y-10),cv2.FONT_HERSHEY_DUPLEX,0.8,(255,255,255),2)
        
    cv2.imshow('MASK DETECTOR',img)
    key=cv2.waitKey(1)
    if(key==27):
        break
cv2.destroyAllWindows()
source.release()


## Thank You